In [2]:
import librosa
import librosa.feature
import librosa.display
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.utils.np_utils import to_categorical

In [3]:
def display_mfcc(song):
    y, sample_rate = librosa.load(song)
    plt.plot(range(0, len(y)), y)
    plt.show()
    print(sample_rate)
    print(y)
    mfcc = librosa.feature.mfcc(y)

    fur = librosa.stft(y)
    S_db = librosa.amplitude_to_db(np.abs(fur), ref=np.max)
    plt.figure()
    librosa.display.specshow(S_db)
    plt.colorbar()

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfcc, x_axis='time', y_axis='mel')
    plt.colorbar()
    plt.title(song)
    plt.tight_layout()
    plt.show()

In [4]:
def extract_features_song(f):
    y, _ = librosa.load(f)

    # get Mel-frequency cepstral coefficients
    mfcc = librosa.feature.mfcc(y)
    # normalize values between -1,1 (divide by max)
    mfcc /= np.amax(np.absolute(mfcc))

    return np.ndarray.flatten(mfcc)[:25000]

In [5]:
def generate_features_and_labels():
    all_features = []
    all_labels = []

    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    #genres = ['blues']

    for genre in genres:
        sound_files = glob.glob('genres/'+genre+'/*.au')
        print('Processing %d songs in %s genre...' % (len(sound_files), genre))
        for f in sound_files:
            features = extract_features_song(f)
            all_features.append(features)
            all_labels.append(genre)

    # convert labels to one-hot encoding
    label_uniq_ids, label_row_ids = np.unique(all_labels, return_inverse=True)
    label_row_ids = label_row_ids.astype(np.int32, copy=False)
    onehot_labels = to_categorical(label_row_ids, len(label_uniq_ids))
    return np.stack(all_features), onehot_labels


In [6]:
features, labels = generate_features_and_labels()

Processing 100 songs in blues genre...
Processing 100 songs in classical genre...
Processing 100 songs in country genre...
Processing 100 songs in disco genre...
Processing 100 songs in hiphop genre...
Processing 100 songs in jazz genre...
Processing 100 songs in metal genre...
Processing 100 songs in pop genre...
Processing 100 songs in reggae genre...
Processing 100 songs in rock genre...


In [7]:
#display_mfcc('song2.wav')
print(np.shape(features))
print(np.shape(labels))

training_split = 0.8

# last column has genre, turn it into unique ids
all_data = np.column_stack((features, labels))

np.random.shuffle(all_data)
split_dx = int(len(all_data) * training_split)
train, test = all_data[:split_dx, :], all_data[split_dx:, :]

print(np.shape(train))
print(np.shape(test))

train_input = train[:, :-10]
train_labels = train[:, -10:]

test_input = test[:, :-10]
test_labels = test[:, -10:]

print(np.shape(train_input))
print(np.shape(train_labels))


model = Sequential([
    Flatten(input_shape=np.shape(train_input[0])),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax'),
    ])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

model.fit(train_input, train_labels, epochs=10, batch_size=32)

loss, acc = model.evaluate(test_input, test_labels, batch_size=32)

print("Done!")
print("Loss: %.4f, accuracy: %.4f" % (loss, acc))

(1000, 25000)
(1000, 10)
(800, 25010)
(200, 25010)
(800, 25000)
(800, 10)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25000)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12800512  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 12,805,642
Trainable params: 12,805,642
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
25/25 [==============================] - 0s 9ms/step - loss: 2.5577 - accuracy: 0.3000

In [ ]:
dura = extract_features_song('dura.au')
print(model.predict(dura))